In [8]:
import os
import sys
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
import toolsets.spectra_operations as so
from toolsets.file_io import get_file_list, readin_peak_list, prepare_sample_list
from toolsets.ff_droup import process_mzml, get_EIC_list, EIC
from toolsets.feature_alignment import initilize_pmz_rt_list, find_feature, find_istd, clean_bad_features
from feature_normalization.rt_normalization import rt_normalization
from tqdm import tqdm
# from toolsets.feature_alignment import
from toolsets.search import quick_search_values, string_search
# from feature_finding.feature_finding_alphapept import find_feature

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
peak_list_dir = '/Volumes/Natural Product Chemistry/Working Space/Fanzhou/20230607_NQ_Development_H2Lv2/peak_list/pos'
istd_info = pd.read_csv(os.path.join('/Volumes/Natural Product Chemistry/Working Space/Fanzhou/20230607_NQ_Development_H2Lv2', 'istd_info.csv'))
mzml_dir ='/Volumes/Natural Product Chemistry/Working Space/Fanzhou/20230607_NQ_Development_H2Lv2/mzml/pos'

In [13]:
istd_info

,compound_name,mono_mass,RT_suggested,Precursor m/z
0,C4-APS,215.061614,1.18,216.068890
1,C5-APS,229.077265,2.13,230.084541
2,C6-APS,243.092915,2.84,244.100191
3,C7-APS,257.108565,3.42,258.115841
4,C8-APS,271.124215,4.00,272.131491
5,C9-APS,285.139865,4.46,286.147141
6,C10-APS,299.155515,4.91,300.162791
7,C11-APS,313.171165,5.28,314.178441
8,C12-APS,327.186815,5.68,328.194091
9,C13-APS,341.202465,6.03,342.209741


In [5]:
all_lst = get_file_list(peak_list_dir, '.txt')
fractions, qcs, blks = prepare_sample_list(all_lst)

In [6]:
rt_seed =qcs[0]

In [46]:
normalized_folder ='/Volumes/Natural Product Chemistry/Working Space/Fanzhou/20230607_NQ_Development_H2Lv2/normalized_peak_list/pos'


In [48]:
rt_report, intensity_report = rt_normalization(all_lst, rt_seed, istd_info, peak_list_dir, normalized_folder)

100%|██████████| 64/64 [04:23<00:00,  4.11s/it]


In [43]:
intensity_report

,compound_name,Precursor m/z,Blank_01_P,Blank_03_P,Blank_05_P,Blank_06_P,Blank07_P,C18_X0_P,C18_X1_P,C18_X2_P,...,Si_X17_P,Si_X18_P,Si_X2_P,Si_X3_P,Si_X4_P,Si_X5_P,Si_X6_P,Si_X7_P,Si_X8_P,Si_X9_P
0,C4-APS,216.068890,3.781587e+05,3.613272e+05,3.155033e+05,2.869716e+05,1.876476e+05,3.884992e+05,2.701241e+05,1.678826e+05,...,304734.500,2.867443e+05,3.580019e+05,3.479255e+05,1.539706e+05,259428.250,2.620190e+05,3.254392e+05,302175.625,317076.250
1,C5-APS,230.084541,1.088050e+06,1.135869e+06,9.938193e+05,9.554573e+05,7.485117e+05,9.769071e+05,8.991911e+05,6.390338e+05,...,993212.000,9.715478e+05,1.079947e+06,9.359160e+05,7.627746e+05,693395.625,8.001813e+05,8.786748e+05,805933.750,799783.500
2,C6-APS,244.100191,1.735041e+06,1.705419e+06,1.529028e+06,1.501010e+06,1.318973e+06,1.568732e+06,1.715274e+06,1.444545e+06,...,1577235.875,1.492449e+06,1.592284e+06,1.399875e+06,1.213614e+06,1168062.375,9.805344e+05,1.036658e+06,1106075.375,1163224.375
3,C7-APS,258.115841,2.599787e+06,2.389187e+06,2.168283e+06,2.132862e+06,2.190769e+06,2.274721e+06,2.452437e+06,2.260113e+06,...,2240836.500,2.111489e+06,2.357718e+06,2.128141e+06,1.978900e+06,2049826.000,2.025782e+06,1.947493e+06,1896480.875,2001679.750
4,C8-APS,272.131491,3.158670e+06,3.081789e+06,2.937429e+06,2.892142e+06,2.938322e+06,2.957386e+06,3.148137e+06,2.898418e+06,...,2969532.750,2.841816e+06,3.043720e+06,2.698523e+06,2.734577e+06,2826243.500,2.900040e+06,2.816929e+06,2738295.750,2740549.750
5,C9-APS,286.147141,4.038155e+06,3.747327e+06,3.497249e+06,3.499696e+06,3.676590e+06,3.695419e+06,3.847768e+06,3.823006e+06,...,3530315.250,3.504540e+06,3.782805e+06,3.458427e+06,3.255558e+06,3227547.500,3.429204e+06,3.320554e+06,3306515.000,3041449.250
6,C10-APS,300.162791,4.182901e+06,4.024702e+06,3.666486e+06,3.529744e+06,3.985736e+06,3.882978e+06,3.907638e+06,3.925286e+06,...,3716882.750,3.737624e+06,3.971896e+06,3.587894e+06,3.648882e+06,3305637.250,3.376282e+06,3.487328e+06,3582794.750,3396769.000
7,C11-APS,314.178441,4.206842e+06,4.167540e+06,3.681939e+06,3.553012e+06,3.992314e+06,3.858511e+06,4.344734e+06,3.737850e+06,...,3761563.750,3.579442e+06,4.022950e+06,3.385258e+06,3.319468e+06,3359830.250,3.292014e+06,3.569781e+06,3627840.000,3442885.000
8,C12-APS,328.194091,5.037327e+06,4.681847e+06,4.137763e+06,4.024459e+06,4.247660e+06,4.367570e+06,4.599154e+06,4.100811e+06,...,4083201.000,3.972204e+06,4.269070e+06,3.785298e+06,3.704217e+06,3610325.500,3.915956e+06,3.883396e+06,3866427.500,3920202.500
9,C13-APS,342.209741,5.498324e+06,5.394031e+06,4.941260e+06,4.831060e+06,4.969449e+06,5.387327e+06,5.470496e+06,5.272122e+06,...,5385141.500,4.949184e+06,5.301538e+06,4.484334e+06,4.783722e+06,4684735.000,4.521060e+06,4.603028e+06,4754058.500,4677592.000


# Alignment

In [52]:
from toolsets.feature_alignment import initilize_pmz_rt_list, find_feature, find_istd, clean_bad_features, align

In [55]:
master_list = initilize_pmz_rt_list([os.path.join(normalized_folder, x+'.csv') for x in qcs], mode = 'inclusive')

length_of_master list:  21797


100%|██████████| 21244/21244 [00:34<00:00, 623.49it/s]

length_of updated master list:  73042


In [ ]:
alignment = align(file_paths=[os.path.join(normalized_folder, x+'.csv') for x in all_lst], master_list_input=master_list)

 64%|██████▍   | 41/64 [3:15:52<1:40:37, 262.52s/it]